In [1]:
import asyncio

from metagpt.actions import Action, UserRequirement
from metagpt.logs import logger
from metagpt.roles import Role
from metagpt.schema import Message
from metagpt.environment import Environment

from metagpt.const import MESSAGE_ROUTE_TO_ALL

2025-01-24 11:26:01.729 | INFO     | metagpt.const:get_metagpt_package_root:29 - Package root set to e:\wow-agent\notebook


In [2]:
classroom = Environment()

In [3]:
class Student(Role):

    name: str = "xiaoming"
    profile: str = "Student"

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.set_actions([WritePoem])
        self._watch([UserRequirement, ReviewPoem])

    async def _act(self) -> Message:
        logger.info(f"{self._setting}: ready to {self.rc.todo}")
        todo = self.rc.todo

        msg = self.get_memories()  # 获取所有记忆
        # logger.info(msg)
        poem_text = await WritePoem().run(msg)
        logger.info(f'student : {poem_text}')
        msg = Message(content=poem_text, role=self.profile,
                      cause_by=type(todo))

        return msg

class Teacher(Role):

    name: str = "laowang"
    profile: str = "Teacher"

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.set_actions([ReviewPoem])
        self._watch([WritePoem])

    async def _act(self) -> Message:
        logger.info(f"{self._setting}: ready to {self.rc.todo}")
        todo = self.rc.todo

        msg = self.get_memories()  # 获取所有记忆
        poem_text = await ReviewPoem().run(msg)
        logger.info(f'teacher : {poem_text}')
        msg = Message(content=poem_text, role=self.profile,
                      cause_by=type(todo))

        return msg


In [4]:
class WritePoem(Action):

    name: str = "WritePoem"

    PROMPT_TEMPLATE: str = """
    Here is the historical conversation record : {msg} .
    Write a poem about the subject provided by human, Return only the content of the generated poem with NO other texts.
    If the teacher provides suggestions about the poem, revise the student's poem based on the suggestions and return.
    your poem:
    """

    async def run(self, msg: str):

        prompt = self.PROMPT_TEMPLATE.format(msg = msg)

        rsp = await self._aask(prompt)

        return rsp

class ReviewPoem(Action):

    name: str = "ReviewPoem"

    PROMPT_TEMPLATE: str = """

    Here is the historical conversation record : {msg} .
    Check student-created poems about the subject provided by human and give your suggestions for revisions. You prefer poems with elegant sentences and retro style.
    Return only your comments with NO other texts.
    your comments:
    """

    async def run(self, msg: str):

        prompt = self.PROMPT_TEMPLATE.format(msg = msg)

        rsp = await self._aask(prompt)

        return rsp

In [5]:
async def main(topic: str, n_round=3):

    classroom.add_roles([Student(), Teacher()])

    classroom.publish_message(
        Message(role="Human", content=topic, cause_by=UserRequirement,
                send_to='' or MESSAGE_ROUTE_TO_ALL),
        peekable=False,
    )

    while n_round > 0:
        # self._save()
        n_round -= 1 #如果n_round = 1 ，就只有学生写诗、然后老师没办法进行review
        logger.debug(f"max {n_round=} left.")

        await classroom.run()
    return classroom.history

await main(topic='写一副2025年农历蛇年写一首审计工作的辞旧迎新的诗歌')

2025-01-24 11:26:06.092 | INFO     | __main__:_act:12 - xiaoming(Student): ready to WritePoem


春回大地蛇年到，  
审计新篇待书写。  
旧账清算迎新春，  
舞动长蛇展宏图。

数字间寻真相，  
报表中见真章。  
往昔错漏皆成过，  
今朝审计铸辉煌。

蛇年春风送暖意，  
审计人心向阳光。  
携手共进新时代，  
蛇舞审计映辉煌。


2025-01-24 11:26:07.886 | WARNING  | metagpt.provider.openai_api:_calc_usage:244 - usage calculation failed: num_tokens_from_messages() is not implemented for model Qwen2.5-32B-Instruct-AWQ. See https://cookbook.openai.com/examples/how_to_count_tokens_with_tiktoken for information on how messages are converted to tokens.
2025-01-24 11:26:07.888 | INFO     | metagpt.utils.cost_manager:update_cost:108 - prompt_tokens: 0, completion_tokens: 0
2025-01-24 11:26:07.889 | INFO     | __main__:_act:18 - student : 春回大地蛇年到，  
审计新篇待书写。  
旧账清算迎新春，  
舞动长蛇展宏图。

数字间寻真相，  
报表中见真章。  
往昔错漏皆成过，  
今朝审计铸辉煌。

蛇年春风送暖意，  
审计人心向阳光。  
携手共进新时代，  
蛇舞审计映辉煌。
2025-01-24 11:26:07.891 | INFO     | __main__:_act:35 - laowang(Teacher): ready to ReviewPoem


这首诗展现了审计工作的严谨与新年的希望，但为了更符合典雅与复古的风格，可以考虑以下修改建议：

- 将“审计新篇待书写”改为“审计新篇待铺展”，以增加古风韵味。
- 将“旧账清算迎新春”改为“旧账清算迎春归”，以增加诗意。
- 将“舞动长蛇展宏图”改为“长蛇舞动展宏图”，使句子更加流畅。
- 将“往昔错漏皆成过”改为“往昔错漏皆成昨”，以增加古风韵味。
- 将“今朝审计铸辉煌”改为“今朝审计铸华章”，以增加诗意。
- 将“蛇年春风送暖意”改为“蛇年春风送暖意，蛇舞轻盈报佳音”，以增加画面感。
- 将“审计人心向阳光”改为“审计人心向阳春”，以增加诗意。
- 将“携手共进新时代”改为“携手共进新时代，审计春风暖人心”，以增加画面感。

修改后的诗歌如下：
春回大地蛇年到，
审计新篇待铺展。
旧账清算迎春归，
长蛇舞动展宏图。

数字间寻真相，
报表中见真章。
往昔错漏皆成昨，
今朝审计铸华章。

蛇年春风送暖意，蛇舞轻盈报佳音，
审计人心向阳春

2025-01-24 11:26:12.692 | WARNING  | metagpt.provider.openai_api:_calc_usage:244 - usage calculation failed: num_tokens_from_messages() is not implemented for model Qwen2.5-32B-Instruct-AWQ. See https://cookbook.openai.com/examples/how_to_count_tokens_with_tiktoken for information on how messages are converted to tokens.
2025-01-24 11:26:12.692 | INFO     | metagpt.utils.cost_manager:update_cost:108 - prompt_tokens: 0, completion_tokens: 0
2025-01-24 11:26:12.693 | INFO     | __main__:_act:40 - teacher : 这首诗展现了审计工作的严谨与新年的希望，但为了更符合典雅与复古的风格，可以考虑以下修改建议：

- 将“审计新篇待书写”改为“审计新篇待铺展”，以增加古风韵味。
- 将“旧账清算迎新春”改为“旧账清算迎春归”，以增加诗意。
- 将“舞动长蛇展宏图”改为“长蛇舞动展宏图”，使句子更加流畅。
- 将“往昔错漏皆成过”改为“往昔错漏皆成昨”，以增加古风韵味。
- 将“今朝审计铸辉煌”改为“今朝审计铸华章”，以增加诗意。
- 将“蛇年春风送暖意”改为“蛇年春风送暖意，蛇舞轻盈报佳音”，以增加画面感。
- 将“审计人心向阳光”改为“审计人心向阳春”，以增加诗意。
- 将“携手共进新时代”改为“携手共进新时代，审计春风暖人心”，以增加画面感。

修改后的诗歌如下：
春回大地蛇年到，
审计新篇待铺展。
旧账清算迎春归，
长蛇舞动展宏图。

数字间寻真相，
报表中见真章。
往昔错漏皆成昨，
今朝审计铸华章。

蛇年春风送暖意，蛇舞轻盈报佳音，
审计人心向阳春。
携手共进新时代，审计春风暖人心。
2025-01-24 11:26:12.696 | INFO     | __main_

。
携手共进新时代，审计春风暖人心。
春回大地蛇年到，  
审计新篇待铺展。  
旧账清算迎春归，  
长蛇舞动展宏图。

数字间寻真相，  
报表中见真章。  
往昔错漏皆成昨，  
今朝审计铸华章。

蛇年春风送暖意，蛇舞轻盈报佳音，  
审计人心向阳春。  
携手共进新时代，审计春风暖人心

2025-01-24 11:26:14.701 | WARNING  | metagpt.provider.openai_api:_calc_usage:244 - usage calculation failed: num_tokens_from_messages() is not implemented for model Qwen2.5-32B-Instruct-AWQ. See https://cookbook.openai.com/examples/how_to_count_tokens_with_tiktoken for information on how messages are converted to tokens.
2025-01-24 11:26:14.702 | INFO     | metagpt.utils.cost_manager:update_cost:108 - prompt_tokens: 0, completion_tokens: 0
2025-01-24 11:26:14.703 | INFO     | __main__:_act:18 - student : 春回大地蛇年到，  
审计新篇待铺展。  
旧账清算迎春归，  
长蛇舞动展宏图。

数字间寻真相，  
报表中见真章。  
往昔错漏皆成昨，  
今朝审计铸华章。

蛇年春风送暖意，蛇舞轻盈报佳音，  
审计人心向阳春。  
携手共进新时代，审计春风暖人心。


。


'\nHuman: 写一副2025年农历蛇年写一首审计工作的辞旧迎新的诗歌\nStudent: 春回大地蛇年到，  \n审计新篇待书写。  \n旧账清算迎新春，  \n舞动长蛇展宏图。\n\n数字间寻真相，  \n报表中见真章。  \n往昔错漏皆成过，  \n今朝审计铸辉煌。\n\n蛇年春风送暖意，  \n审计人心向阳光。  \n携手共进新时代，  \n蛇舞审计映辉煌。\nTeacher: 这首诗展现了审计工作的严谨与新年的希望，但为了更符合典雅与复古的风格，可以考虑以下修改建议：\n\n- 将“审计新篇待书写”改为“审计新篇待铺展”，以增加古风韵味。\n- 将“旧账清算迎新春”改为“旧账清算迎春归”，以增加诗意。\n- 将“舞动长蛇展宏图”改为“长蛇舞动展宏图”，使句子更加流畅。\n- 将“往昔错漏皆成过”改为“往昔错漏皆成昨”，以增加古风韵味。\n- 将“今朝审计铸辉煌”改为“今朝审计铸华章”，以增加诗意。\n- 将“蛇年春风送暖意”改为“蛇年春风送暖意，蛇舞轻盈报佳音”，以增加画面感。\n- 将“审计人心向阳光”改为“审计人心向阳春”，以增加诗意。\n- 将“携手共进新时代”改为“携手共进新时代，审计春风暖人心”，以增加画面感。\n\n修改后的诗歌如下：\n春回大地蛇年到，\n审计新篇待铺展。\n旧账清算迎春归，\n长蛇舞动展宏图。\n\n数字间寻真相，\n报表中见真章。\n往昔错漏皆成昨，\n今朝审计铸华章。\n\n蛇年春风送暖意，蛇舞轻盈报佳音，\n审计人心向阳春。\n携手共进新时代，审计春风暖人心。\nStudent: 春回大地蛇年到，  \n审计新篇待铺展。  \n旧账清算迎春归，  \n长蛇舞动展宏图。\n\n数字间寻真相，  \n报表中见真章。  \n往昔错漏皆成昨，  \n今朝审计铸华章。\n\n蛇年春风送暖意，蛇舞轻盈报佳音，  \n审计人心向阳春。  \n携手共进新时代，审计春风暖人心。'